In [ ]:
import pandas as pd
import yaml
from sqlalchemy import create_engine
from datetime import timedelta

## Database Connection

In [ ]:
with open('config.yml', 'r') as f: #Abrir el archivo en modo de  lectura
    config = yaml.safe_load(f) # Crear un diccionario con lo que hay en el archivo
    config_db_etl = config['bodega'] #Obtener solo la configuración de la bodega
    config_db = config["fuente"] #Obtener solo la configuración de la bodega

In [ ]:
# Construct the database URL
url_db_etl = (f"{config_db_etl['driver']}://{config_db_etl['user']}:{config_db_etl['password']}@{config_db_etl['host']}:"
           f"{config_db_etl['port']}/{config_db_etl['db']}")
url_db = (f"{config_db['driver']}://{config_db['user']}:{config_db['password']}@{config_db['host']}:"
           f"{config_db['port']}/{config_db['db']}")

In [ ]:
# Create the SQLAlchemy Engine
etl_conn = create_engine(url_db_etl)
olap_conn = create_engine(url_db)

## Extraction


In [ ]:
dim_fase = pd.read_sql_table('dim_fase_servicio', etl_conn) 
fecha = pd.read_sql_table('dim_fecha', etl_conn)
hora = pd.read_sql_table('dim_hora', etl_conn)
mensajero = pd.read_sql_table('dim_mensajero', etl_conn)
estado_servicio = pd.read_sql_table("mensajeria_estadosservicio",url_db)
servicio = pd.read_sql_table("mensajeria_servicio",url_db)

In [ ]:
servicio = servicio[["id", "fecha_solicitud", "hora_solicitud", "mensajero_id"]]
servicio.head(5)

In [ ]:
mensajero.head(3)

In [ ]:
estado_servicio.head(5)

In [ ]:
estado_servicio.drop(columns=["foto","observaciones","es_prueba","foto_binary"], inplace=True)


In [ ]:
estado_servicio[estado_servicio['estado_id'].isin([1, 2, 3, 4, 5, 6])]['estado_id'].value_counts()
estado_servicio.head(5)

In [ ]:
# Convertir la columna fecha a formato datetime si no lo está
estado_servicio['fecha'] = pd.to_datetime(estado_servicio['fecha'])

# Filtrar el DataFrame para los estados diferentes de 1
df_no_estado_1 = estado_servicio[estado_servicio['estado_id'] != 1]

# Agrupar por 'id' y 'estado_id', y seleccionar la fila con la fecha más reciente
df_no_estado_1_max_fecha = df_no_estado_1.loc[df_no_estado_1.groupby(['id', 'estado_id'])['fecha'].idxmax()]

# Filtrar el DataFrame para los registros con estado_id == 1
df_estado_1 = estado_servicio[estado_servicio['estado_id'] == 1]

# Concatenar ambos DataFrames: los registros con estado_id == 1 y los seleccionados por fecha para los demás
estado_servicio_final = pd.concat([df_estado_1, df_no_estado_1_max_fecha])

# Ordenar si lo deseas por id y estado_id (opcional)
estado_servicio_final = estado_servicio_final.sort_values(by=['id'])


In [ ]:
# Ajustar el número de filas y columnas a mostrar
pd.set_option('display.max_rows', None)  # Muestra todas las filas
pd.set_option('display.max_columns', None)  # Muestra todas las columnas

# Ahora imprime el DataFrame completo
print(estado_servicio_final)

In [ ]:
total_nan = estado_servicio_final.isna().sum()

# Asegurarnos de que las columnas de fecha y hora son de tipo string
estado_servicio_final['fecha'] = estado_servicio_final['fecha'].astype(str)
estado_servicio_final['hora'] = estado_servicio_final['hora'].astype(str)

# Truncar los milisegundos (decimales) de la columna 'hora' si existen
estado_servicio_final['hora'] = estado_servicio_final['hora'].str.split('.').str[0]

# Convertir las columnas de fecha y hora en una única columna de tipo datetime
estado_servicio_final['datetime'] = pd.to_datetime(estado_servicio_final['fecha'] + ' ' + estado_servicio_final['hora'], errors='coerce')

# Ordenar el DataFrame por 'servicio_id', 'estado_id' y 'datetime' de forma descendente
estado_servicio_final = estado_servicio_final.sort_values(by=['servicio_id', 'estado_id', 'datetime'], ascending=[True, True, False])

# Mostrar el resultado
estado_servicio_final.head()



In [ ]:
print(len(estado_servicio_final))

## Transformation

### Hour Process

In [ ]:
#estado_servicio_final['hora'] = estado_servicio_final['hora'].astype(str)

# Truncar los milisegundos (decimales) de la columna 'hora' si existen
#estado_servicio_final['hora'] = estado_servicio_final['hora'].str.split('.').str[0]

# Convertir las columnas de fecha y hora en una única columna de tipo datetime
#estado_servicio_final['hora'] = pd.to_datetime(estado_servicio_final['hora'], errors='coerce')

#estado_servicio_final.head(10)

### Date Process

In [ ]:
mensajero.rename(columns={'id':'mensajero_id'}, inplace=True)
estado_servicio_final['fecha'] = pd.to_datetime(estado_servicio_final['fecha'], errors='coerce')

estado_servicio_final = pd.merge(
    estado_servicio_final,
    fecha[["date", "key_fecha"]],
    left_on="fecha",
    right_on="date",
    how="left"
)

servicio = pd.merge(servicio, mensajero[["mensajero_id", "key_mensajero"]], left_on="mensajero_id", right_on="mensajero_id", how="left")
estado_servicio_final.head(5)

In [ ]:
len(servicio)
len(estado_servicio)

### Phase Process

In [ ]:
# dim_fase.rename(columns={'id':'dim_estado_id'}, inplace=True)
# estado_servicio = pd.merge(estado_servicio, dim_fase[["dim_estado_id", "key_fase_servicio"]], left_on="estado_id", right_on="dim_estado_id", how="left")
# estado_servicio

In [ ]:
fase_iniciado = estado_servicio_final[estado_servicio_final['estado_id'] == 1]
fase_mensajero_asignado = estado_servicio_final[estado_servicio_final['estado_id'] == 2]
fase_recogido_mensajero = estado_servicio_final[estado_servicio_final['estado_id'] == 4]
fase_entregado_destino = estado_servicio_final[estado_servicio_final['estado_id'] == 5]
fase_terminado = estado_servicio_final[estado_servicio_final['estado_id'] == 6]

fase_iniciado.columns

In [ ]:

fase_iniciado.rename(columns={'fecha':'fecha_inicio', 'hora':'hora_inicio', 'key_fecha':'key_fecha_inicio'}, inplace=True)
fase_iniciado = fase_iniciado[['servicio_id','fecha_inicio','hora_inicio','key_fecha_inicio','estado_id']]
fase_iniciado.rename(columns={'servicio_id':'id'}, inplace=True)

fase_mensajero_asignado.rename(columns={'fecha':'fecha_mensajero_asignado', 'hora':'hora_mensajero_asignado', 'key_fecha':'key_fecha_mensajero_asignado'}, inplace=True)
fase_mensajero_asignado = fase_mensajero_asignado[['servicio_id','fecha_mensajero_asignado','hora_mensajero_asignado','key_fecha_mensajero_asignado']]
fase_mensajero_asignado.rename(columns={'servicio_id':'id'}, inplace=True)

fase_recogido_mensajero.rename(columns={'fecha':'fecha_recogido_mensajero', 'hora':'hora_recogido_mensajero','key_fecha':'key_fecha_recogido_mensajero'}, inplace=True)
fase_recogido_mensajero = fase_recogido_mensajero[['servicio_id','fecha_recogido_mensajero','hora_recogido_mensajero','key_fecha_recogido_mensajero']]
fase_recogido_mensajero.rename(columns={'servicio_id':'id'}, inplace=True)

fase_entregado_destino.rename(columns={'fecha':'fecha_entregado_destino', 'hora':'hora_entregado_destino','key_fecha':'key_fecha_entregado_destino'}, inplace=True)
fase_entregado_destino = fase_entregado_destino[['servicio_id','fecha_entregado_destino','hora_entregado_destino','key_fecha_entregado_destino']]
fase_entregado_destino.rename(columns={'servicio_id':'id'}, inplace=True)

fase_terminado.rename(columns={'fecha':'fecha_terminado', 'hora':'hora_terminado','key_fecha':'key_fecha_terminado'}, inplace=True)
fase_terminado = fase_terminado[['servicio_id','fecha_terminado','hora_terminado','key_fecha_terminado']]
fase_terminado.rename(columns={'servicio_id':'id'}, inplace=True)

In [ ]:
print(fase_iniciado['id'].duplicated().sum())  # Muestra cuántos duplicados hay
print(fase_mensajero_asignado['id'].duplicated().sum())
print(fase_recogido_mensajero['id'].duplicated().sum())
print(fase_entregado_destino['id'].duplicated().sum())
print(fase_terminado['id'].duplicated().sum())

In [ ]:
duplicados = fase_iniciado[fase_iniciado['id'].duplicated(keep=False)]
duplicados.head(10)

In [ ]:
duplicados.head(10)

In [ ]:
fase_mensajero_asignado.head(10)

In [ ]:
# fase_iniciado = fase_iniciado.drop_duplicates(subset='id')
# fase_mensajero_asignado = fase_mensajero_asignado.drop_duplicates(subset='id')
# fase_recogido_mensajero = fase_recogido_mensajero.drop_duplicates(subset='id')
# fase_entregado_destino = fase_entregado_destino.drop_duplicates(subset='id')
# fase_terminado = fase_terminado.drop_duplicates(subset='id')

In [ ]:
len(fase_terminado)

In [ ]:
servicio = pd.merge(servicio, fase_iniciado[['id', 'fecha_inicio', 'hora_inicio', 'key_fecha_inicio']], left_on=['id', 'fecha_solicitud'], right_on=['id', 'fecha_inicio'], how='left')
servicio = pd.merge(servicio, fase_mensajero_asignado[['id','fecha_mensajero_asignado','hora_mensajero_asignado','key_fecha_mensajero_asignado']], left_on="id", right_on="id", how="left")
servicio = pd.merge(servicio, fase_recogido_mensajero[['id','fecha_recogido_mensajero','hora_recogido_mensajero','key_fecha_recogido_mensajero']], left_on="id", right_on="id", how="left")
servicio = pd.merge(servicio, fase_entregado_destino[['id','fecha_entregado_destino','hora_entregado_destino','key_fecha_entregado_destino']], left_on="id", right_on="id", how="left")
servicio = pd.merge(servicio, fase_terminado[['id','fecha_terminado','hora_terminado','key_fecha_terminado']], left_on="id", right_on="id", how="left")
servicio.head(10)

In [ ]:
servicio[servicio['id'] == 26]



In [ ]:
len(servicio)

In [ ]:
# servicio = servicio.drop_duplicates(subset='id')
servicio.groupby(["id"]).size()
len(servicio)

In [ ]:
servicio.groupby(["id","estado_id"]).size()

In [ ]:
servicio[servicio['id'] == 26]

### Calculo días

In [ ]:
servicio['dias_mensajero_asignado'] = (servicio['fecha_mensajero_asignado'] - servicio['fecha_inicio']).dt.days.fillna(0)
servicio['dias_recogido_mensajero'] = (servicio['fecha_recogido_mensajero'] - servicio['fecha_mensajero_asignado']).dt.days.fillna(0)
servicio['dias_entregado_destino'] = (servicio['fecha_entregado_destino'] - servicio['fecha_recogido_mensajero']).dt.days.fillna(0)
servicio['dias_terminado'] = (servicio['fecha_terminado'] - servicio['fecha_entregado_destino']).dt.days.fillna(0)
# Calcular el promedio general de entrega en días y redondear a 2 decimales
servicio['promedio_entrega_general'] = (
    (servicio['dias_mensajero_asignado'] + 
     servicio['dias_recogido_mensajero'] + 
     servicio['dias_entregado_destino'] + 
     servicio['dias_terminado']) / 4
).round(2)
servicio.head(5)

### Cálculo Hora

In [ ]:
# Calcular diferencias en horas, redondear a 2 decimales, llenar valores nulos con 0 y tomar valor absoluto
servicio['horas_mensajero_asignado'] = ((servicio['hora_mensajero_asignado'] - servicio['hora_inicio']).dt.total_seconds() / 3600).round(2).fillna(0).abs()
servicio['horas_recogido_mensajero'] = ((servicio['hora_recogido_mensajero'] - servicio['hora_mensajero_asignado']).dt.total_seconds() / 3600).round(2).fillna(0).abs()
servicio['horas_entregado_destino'] = ((servicio['hora_entregado_destino'] - servicio['hora_recogido_mensajero']).dt.total_seconds() / 3600).round(2).fillna(0).abs()
servicio['horas_terminado'] = ((servicio['hora_terminado'] - servicio['hora_entregado_destino']).dt.total_seconds() / 3600).round(2).fillna(0).abs()

In [ ]:
servicio.head(5)

In [ ]:
servicio.fillna(-1, inplace=True)

## calculo tiempo por fase_servicio

In [ ]:
# # Ordenar por 'servicio_id' y 'fecha'
# estado_servicio = estado_servicio.sort_values(by=['servicio_id', 'fecha'])

# # Asegurarse de que la columna 'hora' esté en formato timedelta
# estado_servicio['hora'] = pd.to_timedelta(estado_servicio['hora'].astype(str))

# # Calcular la diferencia en días entre fases consecutivas dentro de cada servicio
# estado_servicio['dias_de_demora'] = estado_servicio.groupby('servicio_id')['fecha'].transform(lambda x: x.diff().dt.days)

# # asignar el resultado a la columna directamente para evitar errores jeje
# estado_servicio['dias_de_demora'] = estado_servicio['dias_de_demora'].fillna(0)

# # Calcular la diferencia en horas entre fases consecutivas dentro de cada servicio
# estado_servicio['hora_de_demora'] = estado_servicio.groupby('servicio_id')['hora'].transform(lambda x: x.diff())

# # Corregir las horas negativas: Si la hora actual es menor que la anterior, agregar 24 horas
# estado_servicio['hora_de_demora'] = estado_servicio['hora_de_demora'].apply(lambda x: x if x >= pd.Timedelta(0) else x + pd.Timedelta(days=1))

# # Convertir la diferencia en horas
# estado_servicio['hora_de_demora'] = estado_servicio['hora_de_demora'].dt.total_seconds() / 3600

# # Lo mismo para la columna 'hora_de_demora'
# estado_servicio['hora_de_demora'] = estado_servicio['hora_de_demora'].fillna(0)

# # Convertir las columnas 'hora' y 'hora_de_demora' a su formato de horas
# estado_servicio['hora'] = estado_servicio['hora'].dt.total_seconds() / 3600
# estado_servicio['hora_de_demora'] = estado_servicio['hora_de_demora'].apply(lambda x: round(x, 2))  # Redondear a 2 decimales


# estado_servicio.head(20)


## Calculo atributo tiempo promedio general

In [ ]:
# # Convertir 'hora_de_demora' a timedelta si aún no está en el formato adecuado
# estado_servicio['hora_de_demora'] = pd.to_timedelta(estado_servicio['hora_de_demora'], unit='h')

# # Calcular el tiempo total en días considerando solo las fechas, con dos decimales
# estado_servicio['tiempo_total'] = estado_servicio.groupby('servicio_id')['fecha'].transform(lambda x: (x.max() - x.min()).days).round(2)

# # Calcular el tiempo total en horas sumando las horas de demora de cada fase por servicio
# estado_servicio['tiempo_total_horas'] = estado_servicio.groupby('servicio_id')['hora_de_demora'].transform('sum').dt.total_seconds() / 3600
# estado_servicio['tiempo_total_horas'] = estado_servicio['tiempo_total_horas'].round(2)

# # Calcular el número de fases por servicio
# estado_servicio['numero_fases'] = estado_servicio.groupby('servicio_id')['fecha'].transform('count')

# # Calcular el tiempo promedio por fase en días (dividiendo el tiempo total de días entre el número de fases)
# estado_servicio['tiempo_promedio_dias'] = (estado_servicio['tiempo_total'] / estado_servicio['numero_fases']).round(2)

# # Calcular el tiempo promedio en horas por fase con dos decimales (dividiendo el tiempo total de horas entre el número de fases)
# estado_servicio['tiempo_promedio_horas'] = (estado_servicio['tiempo_total_horas'] / estado_servicio['numero_fases']).round(2)

# # Convertir las columnas 'hora' y 'hora_de_demora' a su formato de horas
# estado_servicio['hora_de_demora'] = estado_servicio['hora_de_demora'].dt.total_seconds() / 3600

# # Mostrar los primeros 10 resultados
# estado_servicio.head(10) 


## Eliminar columnas no necesarias

In [ ]:
servicio.drop(columns=['mensajero_id','fecha_solicitud','hora_solicitud','fecha_inicio','hora_inicio','fecha_mensajero_asignado','hora_mensajero_asignado','fecha_recogido_mensajero','hora_recogido_mensajero','fecha_entregado_destino','hora_entregado_destino','fecha_terminado','hora_terminado'], inplace=True)
servicio.head(10)

In [ ]:
servicio[servicio['id']==24]

In [ ]:
len(servicio)

## Load

In [ ]:
servicio.to_sql("hecho_accumulating_snapshot", etl_conn, if_exists="replace", index_label="key_servicio_fase") 